In [ ]:
import numpy as np
import mdtraj as md

In [ ]:
t = md.load('coordinates.pdb')

origin = 1 #index for first origin atom
atomspermol = 6 #how many atoms per mole
nmols = 1728 #total number of molecules
cutoff = 5 #cutoff in angstroms

coords = t.xyz*10 #defines coordinates.  for some reason mdtraj divides them by 0, so I fixed that.

#computes distances from two sets of xyz coordinates
def distance(p1,p2):
    x1,y1,z1 = p1
    x2,y2,z2 = p2
    return np.sqrt((x2-x1)**2+(y2-y1)**2+(z2-z1)**2)

neighbors = np.zeros((nmols,nmols))

for i in range(nmols):
    for j in range(i-1):
        if distance(coords[0][i*atomspermol+origin],coords[0][j*atomspermol+origin]) <= cutoff:
            neighbors[i][j] = 1
            neighbors[j][i] = 1

nneighbors = np.sum(neighbors,axis=1)

In [ ]:
print(nneighbors)

f = open('OPs.cop','r')
g = open('normalOPs','w')

orderparameters = []
for x in f:
    orderparameters.append(x)
    
f.close()

bondops = np.array(orderparameters[(10+nmols):(10+2*nmols)],dtype=float)
normalops = bondops/nneighbors
np.savetxt('newOPs.txt',normalops)